<a href="https://colab.research.google.com/github/Paulina9555/039_apriori2/blob/main/039_apriori2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Spis treści:
1. [Import bibliotek](#0)
2. [Wygenerowanie danych](#1)
3. [Przygotowanie danych](#2)
4. [Kodowanie transakcji](#3)
5. [Algorytm Apriori](#4)

### <a name='0'></a> Import bibliotek

In [1]:
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:2f}')

### <a name='1'></a> Wygenerowanie danych

In [2]:
!wget = https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
!wget = https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv

--2025-09-20 13:05:20--  http://=/
Resolving = (=)... failed: Name or service not known.
wget: unable to resolve host address ‘=’
--2025-09-20 13:05:20--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.0.91, 172.217.12.27, 172.217.15.251, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.0.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2166953 (2.1M) [application/octet-stream]
Saving to: ‘products.csv’

products.csv        100%[===================>]   2.07M  2.18MB/s    in 0.9s    

2025-09-20 13:05:22 (2.18 MB/s) - ‘products.csv’ saved [2166953/2166953]

FINISHED --2025-09-20 13:05:22--
Total wall clock time: 1.9s
Downloaded: 1 files, 2.1M in 0.9s (2.18 MB/s)
--2025-09-20 13:05:22--  http://=/
Resolving = (=)... failed: Name or service not known.
wget: unable to resolve host address ‘=’
--2025-09-20 13:05:22--  https://storage.googl

In [3]:
products = pd.read_csv('products.csv', usecols=['product_id', 'product_name'])
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [4]:
orders = pd.read_csv('orders.csv', usecols=['order_id', 'product_id'])
orders.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


### <a name='2'></a> Przygotowanie danych

In [5]:
data = pd.merge(orders, products, how='inner', on='product_id', sort=True)
data = data.sort_values(by='order_id')
data.head()

,order_id,product_id,product_name
325660,1,13176,Bag of Organic Bananas
1382192,1,49683,Cucumber Kirby
1375139,1,49302,Bulgarian Yogurt
1302365,1,47209,Organic Hass Avocado
588447,1,22035,Organic Whole String Cheese


In [6]:
data.describe()

,order_id,product_id
count,1384617.000000,1384617.000000
mean,1706297.621090,25556.235729
std,989732.648868,14121.272449
min,1.000000,1.000000
25%,843370.000000,13380.000000
50%,1701880.000000,25298.000000
75%,2568023.000000,37940.000000
max,3421070.000000,49688.000000


In [7]:
# rozkład produktów
data['product_name'].value_counts()

,count
product_name,
Banana,18726
Bag of Organic Bananas,15480
Organic Strawberries,10894
Organic Baby Spinach,9784
Large Lemon,8135
...,...
Radiant Infinity Overnight Light Clean Scent Pads With Wings,1
Classic Original Lip Balm SPF 12,1
Sweet & Thick Original BBQ Sauce,1


In [8]:
# liczba transakcji
data['order_id'].nunique()

131209

In [10]:
transactions = data.groupby(by='order_id')['product_name'].apply(lambda name: ','.join(name))
transactions

,product_name
order_id,
1,"Bag of Organic Bananas,Cucumber Kirby,Bulgaria..."
36,"Grated Pecorino Romano Cheese,Organic Garnet S..."
38,"Green Peas,Bunched Cilantro,Flat Parsley, Bunc..."
96,"Organic Pomegranate Kernels,Organic Blueberrie..."
98,"Organic Ketchup,Queso Fresco,Aluminum Foil,Org..."
...,...
3421049,"Organic Baby Broccoli,Organic Whole Grain Whea..."
3421056,"Tartar Sauce,Homestyle Classics Meatloaf,Spark..."
3421058,"Club Soda Lower Sodium,Classic Britannia Crisp..."


In [11]:
transactions = transactions.str.split(',')
transactions

,product_name
order_id,
1,"[Bag of Organic Bananas, Cucumber Kirby, Bulga..."
36,"[Grated Pecorino Romano Cheese, Organic Garnet..."
38,"[Green Peas, Bunched Cilantro, Flat Parsley, ..."
96,"[Organic Pomegranate Kernels, Organic Blueberr..."
98,"[Organic Ketchup, Queso Fresco, Aluminum Foil,..."
...,...
3421049,"[Organic Baby Broccoli, Organic Whole Grain Wh..."
3421056,"[Tartar Sauce, Homestyle Classics Meatloaf, Sp..."
3421058,"[Club Soda Lower Sodium, Classic Britannia Cri..."


### <a name='3'></a> Kodowanie transakcji

In [13]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()
encoder.fit(transactions)
transactions_encoder = encoder.transform(transactions, sparse=True)
transactions_encoder

<Compressed Sparse Row sparse matrix of dtype 'bool'
	with 1442410 stored elements and shape (131209, 40434)>

In [14]:
transactions_encoder_df = pd.DataFrame(transactions_encoder.toarray(), columns=encoder.columns_)
transactions_encoder_df

,,Apricot & Banana Stage 2 Baby Food,Broad Spectrum SPF 30,Instant,Livermore Valley,Low Sodium Marinara,Premium,Vetiver scent,Whole,#2,...,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water,Lightly Seasoned with Rosemary and Roasted Garlic Family Size Herb Chicken Tortellini
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131205,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131206,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131207,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### <a name='4'></a> Algorytm Apriori

In [18]:
from mlxtend.frequent_patterns import apriori, association_rules

supports = apriori(transactions_encoder_df, min_support=0.01, use_colnames=True)
supports = supports.sort_values(by='support', ascending=False)
supports.head()

,support,itemsets
8,0.143138,(Banana)
7,0.117980,(Bag of Organic Bananas)
76,0.083028,(Organic Strawberries)
41,0.074568,(Organic Baby Spinach)
31,0.062000,(Large Lemon)


In [21]:
rules = association_rules(supports, metric='confidence', min_threshold=0)
rules = rules.iloc[:, [0, 1, 4, 5, 6]]
rules = rules.sort_values(by='lift', ascending=False)
rules.head(15)

,antecedents,consequents,support,confidence,lift
27,(Clementines),( Bag),0.011295,0.523860,36.835557
26,( Bag),(Clementines),0.011295,0.794212,36.835557
22,(Limes),(Large Lemon),0.012156,0.264379,4.264159
23,(Large Lemon),(Limes),0.012156,0.196066,4.264159
19,(Organic Raspberries),(Organic Strawberries),0.012728,0.301118,3.626710
18,(Organic Strawberries),(Organic Raspberries),0.012728,0.153295,3.626710
31,(Large Lemon),(Organic Avocado),0.010281,0.165827,2.936692
30,(Organic Avocado),(Large Lemon),0.010281,0.182076,2.936692
3,(Organic Hass Avocado),(Bag of Organic Bananas),0.018444,0.331825,2.812560
2,(Bag of Organic Bananas),(Organic Hass Avocado),0.018444,0.156331,2.812560
